In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
# 모델
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [2]:
gens = pd.read_csv("../gens.csv")
p_gens = pd.read_csv("../pred.csv")
wf = pd.read_csv("../weather_forecast.csv")

In [3]:
# 발전량 예측량을 모델별로 분리해서 변수로 만들라고 분리하는거
p_gens0 = p_gens.loc[(p_gens["model_id"]==0) & (p_gens["round"]==1)].reset_index()
p_gens1 = p_gens.loc[(p_gens["model_id"]==1) & (p_gens["round"]==1)].reset_index()
p_gens2 = p_gens.loc[(p_gens["model_id"]==2) & (p_gens["round"]==1)].reset_index()
p_gens3 = p_gens.loc[(p_gens["model_id"]==3) & (p_gens["round"]==1)].reset_index()
p_gens4 = p_gens.loc[(p_gens["model_id"]==4) & (p_gens["round"]==1)].reset_index()

In [4]:
# 날씨 예측량을 데이터프레임으로 왜냐면 모델 train 시킬라고
wf_round1 = wf[wf["round"]==1]
wf_round2 = wf[wf["round"]==2]

In [5]:
x_df = pd.DataFrame({"m0": p_gens0["amount"],"m1": p_gens1["amount"],
              "m2": p_gens2["amount"],"m3": p_gens3["amount"],
              "m4": p_gens4["amount"], "time" : p_gens0["time"]})
y_df= gens["amount"]

In [6]:
xy_df = pd.merge(x_df,wf_round1,on="time")
xy_df = pd.merge(xy_df,gens,on="time")
x_df = xy_df.drop(["round","amount"],axis=1)
y_df = xy_df["amount"]

In [7]:
# 독립 변수(X)와 종속 변수(y) 설정
X = x_df[['m0','m1','m2','m3','m4']]
y = y_df

In [8]:
# 'time' 열을 날짜 형식으로 변환
x_df['time'] = pd.to_datetime(x_df['time'])

# 월(month) 정보를 추출하여 'month' 열 생성
x_df['month'] = x_df['time'].dt.month

# 계절별로 데이터 분할
summer_data = x_df[(x_df['month'] >= 6) & (x_df['month'] <= 9)]  # 6월부터 9월까지: 여름
fall_data = x_df[(x_df['month'] >= 10) & (x_df['month'] <= 11)]  # 10월부터 11월까지: 가을
winter_data = x_df[(x_df['month'] >= 12) | (x_df['month'] <= 2)]  # 12월부터 2월까지: 겨울
spring_data = x_df[(x_df['month'] >= 3) & (x_df['month'] <= 5)]  # 3월부터 5월까지: 봄

# 'time' 열을 날짜 형식으로 변환
gens['time'] = pd.to_datetime(gens['time'])

# 월(month) 정보를 추출하여 'month' 열 생성
gens['month'] = gens['time'].dt.month

# 계절별로 데이터 분할
summer_gens = gens[(gens['month'] >= 6) & (gens['month'] <= 9)]  # 여름
fall_gens = gens[(gens['month'] >= 10) & (gens['month'] <= 11)]  # 가을
winter_gens = gens[(gens['month'] >= 12) | (gens['month'] <= 2)]  # 겨울
spring_gens = gens[(gens['month'] >= 3) & (gens['month'] <= 5)]  # 봄

In [9]:
x= fall_data[["m0","m1","m2","m3","m4"]]
y = fall_gens["amount"]
from sklearn.linear_model import LinearRegression

lm = LinearRegression()
lm.fit(x,y)

LinearRegression()

In [1]:
import joblib

In [10]:

joblib.dump(lm,"../model/모델예측량만사용단순회귀계절10-11_r1.pkl")

['../model/모델예측량만사용단순회귀계절9-11_r1.pkl']

# 10-30일껄로 예측 진행

In [4]:
x_pred_ori = pd.read_csv("../data/gen_fcst_10_2023-11-02.csv")
x_pred_ori=x_pred_ori.drop(["time"],axis=1)

In [5]:
x_pred_ori.columns = ["m0","m1","m2","m3","m4"]

In [7]:
lm = joblib.load("../model/모델예측량만사용단순회귀계절10-11_r1.pkl")

In [8]:
pred = lm.predict(x_pred_ori)

In [42]:
import numpy as np

def testMSEnIncentive(pred):
    realGens1030 = np.array([0, 0, 0, 0, 0, 0,
                            0, 4, 20, 56,
                            65, 79, 81, 75,
                            62, 45, 22, 3, 0,
                            0, 0, 0, 0, 0])    
    
    rss = np.power(pred - realGens1030, 2)
    rss1 = rss[6:10]
    rss2 = rss[10:14]
    rss3 = rss[14:19]
    mse1 = sum(rss1/len(rss1))
    mse2 = sum(rss2/len(rss2))
    mse3 = sum(rss3/len(rss3))
    mse = [mse1,mse2,mse3]

    score_df = abs(pred - realGens1030)*100/99
    score = []
    for idx, i in enumerate(score_df):
        if i <=6:
            score.append(realGens1030[idx] * 4)
        elif (i <=8) & (i > 6):
            score.append(realGens1030[idx] * 3)
        else:
            score.append(0)
    score1 = score[6:10]
    score2 = score[10:14]
    score3 = score[14:19]
    final_score = [score1,score2,score3]

    return mse, final_score

In [9]:
pred 

array([-0.4044906 , -0.4044906 , -0.4044906 , -0.4044906 , -0.4044906 ,
       -0.4044906 ,  1.79736311,  8.11522367, 21.95566785, 41.65715269,
       59.1596245 , 70.45822059, 67.03093977, 62.6081862 , 51.56083932,
       32.80954044, 13.37310034,  3.52794144,  0.97342736, -0.4044906 ,
       -0.4044906 , -0.4044906 , -0.4044906 , -0.4044906 ])

In [55]:
testMSEnIncentive(pred)

([39.407835249384284, 43.454038875626786, 18.09839819376539],
 [[0, 16, 60, 0], [260, 237, 243, 225], [248, 135, 88, 12, 0]])